In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
df = pd.read_pickle("clean_train_abstract")
df.shape

(44939, 6)

In [8]:
indexes = []
for x in range(0,len(df)):
    if(len(df['clean_train_abstract'][x])<100):
        indexes.append(x)

In [9]:
df = df.drop(indexes)
df.index = range(len(df.index))

In [16]:
criterion = df['date'].map(lambda x: x.startswith(str(2017)))
df_year = df[criterion]
len(df_year)

2369

In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [18]:
data_words = list(sent_to_words(df_year['clean_train_abstract']))

In [19]:
bigram = gensim.models.Phrases(data_words, min_count=2, threshold=10) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [20]:
data_words_bigrams = make_bigrams(data_words)

In [21]:
id2word = corpora.Dictionary(data_words_bigrams)

In [22]:
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [23]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
id2word=id2word,
num_topics=20,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto',
per_word_topics=True)

In [31]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [32]:
lda_model.show_topics()

[(0,
  '0.047*"render" + 0.017*"locat" + 0.017*"path" + 0.017*"curv" + 0.016*"web_servic" + 0.015*"interpol" + 0.014*"contour" + 0.012*"interconnect" + 0.010*"cmos_technolog" + 0.009*"inerti_particl"'),
 (4,
  '0.047*"secur" + 0.034*"attribut" + 0.029*"attack" + 0.026*"mode" + 0.021*"scheme" + 0.018*"assist" + 0.018*"trust" + 0.017*"polici" + 0.012*"decod" + 0.011*"verif"'),
 (2,
  '0.039*"core" + 0.026*"execut" + 0.025*"parallel" + 0.021*"workload" + 0.020*"runtim" + 0.020*"gpu" + 0.018*"schedul" + 0.017*"db" + 0.016*"degrad" + 0.016*"redund"'),
 (11,
  '0.017*"multipli" + 0.014*"littl" + 0.014*"medium" + 0.013*"tensor" + 0.013*"hard" + 0.012*"amplifi" + 0.012*"distort" + 0.012*"suppli" + 0.012*"overview" + 0.011*"gradient"'),
 (10,
  '0.037*"code" + 0.033*"color" + 0.027*"segment" + 0.022*"sens" + 0.019*"text" + 0.019*"filter" + 0.018*"graphic" + 0.016*"output" + 0.014*"concept" + 0.013*"palett"'),
 (13,
  '0.086*"algorithm" + 0.059*"problem" + 0.044*"optim" + 0.024*"spatial" + 0.021

# pyldavis


**circle의 크기는 비중을 나타냄**

1. 하지만 상대적인 크기를 알기는 힘들다. --> topic1의 크기가 topic2의 크기보다 몇배 더 크다.

2. semantically similarity를 알 수 없음

3. 축의 의미가 없음

In [33]:
vis_data = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis_data)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))
